In [162]:
import json
from functools import partial
from inspect import getsource
from math import log
from sys import argv
from bisect import bisect_left
import re

if __name__ == '__main__':
    map_in_name, script_name = '../cases/case1.json', 'test.py'
    nf = '../input/case1_in.json'
    nf = open(nf, 'r')
    nf = json.load(nf)
    keys, values = sorted(nf[0]), sorted(nf[1])
    
    map_in_file = open(map_in_name)
    origin_mapping = json.load(map_in_file)
    map_in_file.close()

    mmm = sorted(list(origin_mapping.items()),key=lambda x: x[0])

    k,v = zip(*mmm)
    mm = sorted(zip(v,[ i for i in range(len(k)) ]),key=lambda x: x[0])
    v, idx = zip(*mm)
    idx = list(idx)
    
    mark = []
    flag = False
    temp = [ 0, 0, 0 ]
    for i in range(len(k)):
        if i != idx[i]:
            if flag == False:
                temp = [ i, idx[i], 1 ]
                flag = True
            elif flag == True and idx[i]-(temp[1]+temp[2]) == 0:
                temp[2] += 1
            elif flag == True:
                mark += temp
                temp = [ i, idx[i], 1 ]
                flag = True
        else:
            if flag == True:
                mark += temp
                flag = False    
    om = []
    for i in range(0,len(mark),3):
        om.append([mark[i:i+3]])
    
    mark_file = open('mark/'+re.search('case[0-9]+',str(map_in_file)).group(0)+'.mark', 'w')
    mark_file.write(str(om))
    mark_file.close()
    document = """
import json
import hashlib
from sys import argv

_m = {}
nf, of = argv[1], argv[2]
nf = open(nf, 'r')
nf = json.load(nf)
keys, values = sorted(nf[0]), sorted(nf[1])

nm = {{}}
for i in range(len(keys)):
    nm[keys[i]] = values[i]
for i in range(len(_m)):
    for j in range(_m[i][2]):
        nm[keys[_m[i][1]+j]] = values[_m[i][0]+j]
of = open(of, 'w')
of.write(json.dumps(nm))
of.close()
    """.format(str(mark))
    script = open(script_name, 'w')
    script.write(document)
    script.close()

[1, 2, 3, 4]
50161


AttributeError: 'NoneType' object has no attribute 'group'